## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Start URL for Gmaps API call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Run a request to check status of API response
response = requests.get(base_url)
response

<Response [200]>

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Alindao,CF,5.0267,21.2088,77.61,25,16,3.94,few clouds
1,1,Hermanus,ZA,-34.4187,19.2345,63.81,76,0,11.45,clear sky
2,2,Bereznyky,UA,48.5141,23.2198,28.15,95,84,4.94,broken clouds
3,3,Ambulu,ID,-8.3450,113.6058,75.42,86,100,2.30,overcast clouds
4,4,Namatanai,PG,-3.6667,152.4333,81.93,78,93,3.71,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 42
What is the maximum temperature you would like for your trip? 65


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hermanus,ZA,-34.4187,19.2345,63.81,76,0,11.45,clear sky
12,12,Punta Arenas,CL,-53.1500,-70.9167,62.71,55,0,5.75,clear sky
15,15,Gejiu,CN,23.3060,103.1635,43.83,87,97,3.87,overcast clouds
21,21,Bluff,NZ,-46.6000,168.3333,61.81,69,92,6.13,overcast clouds
22,22,Naze,JP,28.3667,129.4833,56.39,76,100,12.17,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
654,654,Racale,IT,39.9631,18.0938,45.84,73,85,16.17,overcast clouds
657,657,Abha,SA,18.2164,42.5053,51.62,87,0,0.00,clear sky
658,658,Arlit,NE,18.7369,7.3853,63.09,15,0,6.33,clear sky
677,677,Jati,PK,24.3539,68.2671,61.54,52,0,5.48,clear sky


In [60]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [61]:
# Verify column with missing values
preferred_cities_df["Country"].isnull().sum()

2

In [48]:
# 4b. Replace whitespace with Nan.
preferred_cities_copy_df = preferred_cities_df.copy()
preferred_cities_copy_df = preferred_cities_copy_df.replace(r'^s*$', float('NaN'), regex = True)

In [57]:
# Drop rows with Nan
clean_df = preferred_cities_df.dropna(how='any')

In [63]:
# Verify count to see if nay nulls remain
clean_df["Country"].isnull().sum()

0

In [76]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hermanus,ZA,63.81,clear sky,-34.4187,19.2345,
12,Punta Arenas,CL,62.71,clear sky,-53.1500,-70.9167,
15,Gejiu,CN,43.83,overcast clouds,23.3060,103.1635,
21,Bluff,NZ,61.81,overcast clouds,-46.6000,168.3333,
22,Naze,JP,56.39,overcast clouds,28.3667,129.4833,
29,Hobart,AU,64.26,broken clouds,-42.8794,147.3294,
30,Ushuaia,AR,64.06,scattered clouds,-54.8000,-68.3000,
33,Mataura,NZ,61.48,overcast clouds,-46.1927,168.8643,
41,Los Llanos De Aridane,ES,56.95,clear sky,28.6585,-17.9182,
43,New Norfolk,AU,63.91,overcast clouds,-42.7826,147.0587,


In [77]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # 6c. Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [78]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [79]:
# Check DF before export
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hermanus,ZA,63.81,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
12,Punta Arenas,CL,62.71,clear sky,-53.1500,-70.9167,Hotel Hain
15,Gejiu,CN,43.83,overcast clouds,23.3060,103.1635,Hongtai Commercial Hotel
21,Bluff,NZ,61.81,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
22,Naze,JP,56.39,overcast clouds,28.3667,129.4833,Shishime Hotel
...,...,...,...,...,...,...,...
654,Racale,IT,45.84,overcast clouds,39.9631,18.0938,B&B Gallipoli Salento al mare
657,Abha,SA,51.62,clear sky,18.2164,42.5053,Abha Hotel
658,Arlit,NE,63.09,clear sky,18.7369,7.3853,Case de Passage De la Prefecture
677,Jati,PK,61.54,clear sky,24.3539,68.2671,Cafe Lala Rehan Restaurant


In [81]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)

In [96]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [97]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))